# TFLite保存与解释与量化

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

2.0.0
sys.version_info(major=3, minor=6, micro=10, releaselevel='final', serial=0)
matplotlib 3.1.2
numpy 1.18.1
pandas 0.25.3
sklearn 0.22.1
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
tflite_dir = os.path.join('tflite_models')
if not os.path.exists(tflite_dir):
    os.mkdir(tflite_dir)

## Keras模型转TFLite

In [4]:
# 加载 keras模型
loaded_keras_model = keras.models.load_model(
    './graph_def_and_weights/fashion_mnist_model.h5')
loaded_keras_model(np.ones((1, 28, 28)))

<tf.Tensor: id=492, shape=(1, 10), dtype=float32, numpy=
array([[0.11120097, 0.03724404, 0.06243855, 0.01093319, 0.03455226,
        0.00780652, 0.1908657 , 0.02144313, 0.49145293, 0.03206269]],
      dtype=float32)>

In [5]:
# 创建转化器
keras_to_tflite_converter = tf.lite.TFLiteConverter.from_keras_model(loaded_keras_model)
# 转化为tflite
keras_tflite = keras_to_tflite_converter.convert()

In [6]:
# 将tflite写入到文件
tflite_file = os.path.join(tflite_dir, 'keras_tflite')

with open(tflite_file, 'wb') as f:
    f.write(keras_tflite)

## Concrete Function 转 TFLite

In [7]:
# 使用tf.function封装 Keras model
run_model = tf.function(lambda x : loaded_keras_model(x))
# 转化为concrete_function
keras_concrete_func = run_model.get_concrete_function(
    tf.TensorSpec(loaded_keras_model.inputs[0].shape, loaded_keras_model.inputs[0].dtype))

In [8]:
keras_concrete_func(tf.constant(np.ones((1, 28, 28), dtype = np.float32)))

<tf.Tensor: id=620, shape=(1, 10), dtype=float32, numpy=
array([[0.11120097, 0.03724404, 0.06243855, 0.01093319, 0.03455226,
        0.00780652, 0.1908657 , 0.02144313, 0.49145293, 0.03206269]],
      dtype=float32)>

In [9]:
# 创建转化器
concrete_func_to_tflite_converter = tf.lite.TFLiteConverter.from_concrete_functions(
    [keras_concrete_func])
# 转化为tflite
concrete_func_tflite = concrete_func_to_tflite_converter.convert()

In [10]:
# 将tflite写入到文件
tflite_file = os.path.join(tflite_dir, 'concrete_func_tflite')

with open(tflite_file, 'wb') as f:
    f.write(concrete_func_tflite)

## SavedModel 转 TFLite

In [11]:
# 创建转化器
saved_model_to_tflite_converter = tf.lite.TFLiteConverter.from_saved_model(
    './keras_saved_graph')
# 转化为tflite
saved_model_tflite = saved_model_to_tflite_converter.convert()

In [12]:
# 将tflite写入到文件
tflite_file = os.path.join(tflite_dir, 'saved_model_tflite')

with open(tflite_file, 'wb') as f:
    f.write(saved_model_tflite)

## 使用tflite interpreter 运行TFLite

In [13]:
# 读取tflite文件
with open('tflite_models/concrete_func_tflite', 'rb') as f:
    concrete_func_tflite = f.read()

# 创建interpreter
interpreter = tf.lite.Interpreter(model_content=concrete_func_tflite)

# 分配内存
interpreter.allocate_tensors()

In [14]:
# 获取输入输出信息
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
print(output_details)

[{'name': 'x', 'index': 10, 'shape': array([ 1, 28, 28]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
[{'name': 'Identity', 'index': 0, 'shape': array([ 1, 10]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]


In [15]:
# 定义输出
input_data = tf.constant(np.ones(input_details[0]['shape'], dtype=np.float32))
interpreter.set_tensor(input_details[0]['index'], input_data)

# 进行预测
interpreter.invoke()

# 获取输出
output_results = interpreter.get_tensor(output_details[0]['index'])
print(output_results)

[[0.11120094 0.03724403 0.06243857 0.01093319 0.03455226 0.00780652
  0.19086568 0.02144315 0.49145293 0.0320627 ]]


## 量化后保存
### keras model

In [16]:
# 创建转化器
keras_to_tflite_converter = tf.lite.TFLiteConverter.from_keras_model(loaded_keras_model)
# 量化
keras_to_tflite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
# 转化为tflite
keras_tflite = keras_to_tflite_converter.convert()

# 将量化后tflite写入到文件
tflite_file = os.path.join(tflite_dir, 'quantized_keras_tflite')
with open(tflite_file, 'wb') as f:
    f.write(keras_tflite)

### concrete func

In [17]:
# 创建转化器
concrete_func_to_tflite_converter = tf.lite.TFLiteConverter.from_concrete_functions(
    [keras_concrete_func])
# 量化
concrete_func_to_tflite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
# 转化为tflite
concrete_func_tflite = concrete_func_to_tflite_converter.convert()

# 将tflite写入到文件
tflite_file = os.path.join(tflite_dir, 'quantized_concrete_func_tflite')
with open(tflite_file, 'wb') as f:
    f.write(concrete_func_tflite)

### Saved Model

In [18]:
# 创建转化器
saved_model_to_tflite_converter = tf.lite.TFLiteConverter.from_saved_model(
    './keras_saved_graph')
# 量化
saved_model_to_tflite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
# 转化为tflite
saved_model_tflite = saved_model_to_tflite_converter.convert()

tflite_file = os.path.join(tflite_dir, 'quantized_saved_model_tflite')
with open(tflite_file, 'wb') as f:
    f.write(saved_model_tflite)

## 使用tflite interpreter 运行量化后的TFLite

In [19]:
# 读取tflite文件
with open('tflite_models/quantized_concrete_func_tflite', 'rb') as f:
    concrete_func_tflite = f.read()

# 创建interpreter
interpreter = tf.lite.Interpreter(model_content=concrete_func_tflite)

# 分配内存
interpreter.allocate_tensors()

In [20]:
# 获取输入输出信息
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
print(output_details)

[{'name': 'x', 'index': 10, 'shape': array([ 1, 28, 28]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
[{'name': 'Identity', 'index': 0, 'shape': array([ 1, 10]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]


In [21]:
# 定义输出
input_data = tf.constant(np.ones(input_details[0]['shape'], dtype=np.float32))
interpreter.set_tensor(input_details[0]['index'], input_data)

# 进行预测
interpreter.invoke()

# 获取输出
output_results = interpreter.get_tensor(output_details[0]['index'])
print(output_results)

[[0.11255115 0.03726469 0.06337408 0.01115514 0.03467338 0.00785631
  0.19097982 0.02149594 0.48883578 0.0318136 ]]
